# Install Dependencies

## External Dependencies

In [2]:
!pip install easy-equities-client

## FrostAura Dependencies

In [3]:
import os
import sys

USE_LOCAL_CODE: bool = True # NOTE: When changing this configuration, be sure to restart the interpreter in order to have sys path updates reflect.
directories_to_move_up: int = 2
current_executing_path: str = os.getcwd()
current_executing_path = current_executing_path.replace('\\', '/')
root_path: str = '/'.join(current_executing_path.split('/')[:-directories_to_move_up])

# Clean up.
%pip uninstall frostaura -y
sys.path = [p for p in sys.path if p != root_path]

if USE_LOCAL_CODE:
    sys.path.append(root_path)
else:
    %pip install -U --no-cache-dir frostaura

Note: you may need to restart the kernel to use updated packages.


## Logging

In [4]:
import logging

logging.basicConfig(level=logging.DEBUG)

# Import Dependencies

In [5]:
from frostaura.data_access import IResourcesDataAccess
from frostaura.data_access import EmbeddedResourcesDataAccess
from frostaura.data_access import IPersonalAssetDataAccess
from frostaura.data_access import EasyEquitiesPersonalAssetDataAccess
from easy_equities_client.accounts.types import Account

# Usage

In [6]:
resource_data_access: IResourcesDataAccess = EmbeddedResourcesDataAccess()
instance: IPersonalAssetDataAccess = EasyEquitiesPersonalAssetDataAccess(resource_data_access=resource_data_access,
                                                                         username='DeanMar09',
                                                                         password='...')

## Get Supported Symbols

In [7]:
supported_symbols: list = instance.get_supported_assets()

print(supported_symbols[:5])

INFO:root:Fetching EasyEquities supported symbols from a static source.
INFO:root:Fetching embedded resource "c:/dev/P/fa.intelligence.notebooks/frostaura/resources/easy_equities_us_stocks.json".


[['10X Genomics Inc', 'TXG'], ['21Vianet Group Inc', 'VNET'], ['2U Inc', 'TWOU'], ['3D Systems Inc', 'DDD'], ['3M Co', 'MMM']]


## Get Accounts & Holdings

In [8]:
accounts: list = instance.__get_accounts__()
usd_account: Account = instance.__get_account_by_name__(account_name='EasyEquities USD')
usd_account_holdings: list = instance.__get_account_holdings__(account_name='EasyEquities USD')
usd_account_valuation: dict = instance.__get_account_valuation__(account_name='EasyEquities USD')

INFO:root:Signing into EasyEquities as "DeanMar09".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): platform.easyequities.io:443
DEBUG:urllib3.connectionpool:https://platform.easyequities.io:443 "POST /Account/SignIn HTTP/1.1" 302 144
DEBUG:urllib3.connectionpool:https://platform.easyequities.io:443 "GET /AccountOverview HTTP/1.1" 200 15112
INFO:root:Signing into EasyEquities as "DeanMar09".
DEBUG:urllib3.connectionpool:https://platform.easyequities.io:443 "POST /Account/SignIn HTTP/1.1" 302 144
DEBUG:urllib3.connectionpool:https://platform.easyequities.io:443 "GET /AccountOverview HTTP/1.1" 200 15112
DEBUG:root:Filtering accounts list by name "EasyEquities USD".
INFO:root:Signing into EasyEquities as "DeanMar09".
DEBUG:urllib3.connectionpool:https://platform.easyequities.io:443 "POST /Account/SignIn HTTP/1.1" 302 144
DEBUG:urllib3.connectionpool:https://platform.easyequities.io:443 "GET /AccountOverview HTTP/1.1" 200 15128
DEBUG:root:Filtering accounts list by name "Ea

In [9]:
usd_account_valuation

{'NetInterestOnCashItems': [{'Label': 'Total Interest on Free Cash',
   'Value': 'R 0.00'},
  {'Label': 'Interest Income', 'Value': 'R 0.00'},
  {'Label': 'Cash Management Fee', 'Value': 'R 0.00'},
  {'Label': 'VAT On Cash Management Fee', 'Value': 'R 0.00'}],
 'AccrualSummaryItems': [{'Label': 'Net Accrual', 'Value': 'R 0.00'}],
 'TopSummary': {'AccountValue': 0.0,
  'AccountCurrency': 'ZAR',
  'AccountNumber': 'EE106679-163510',
  'AccountName': 'EasyEquities ZAR',
  'PeriodMovements': [{'ValueMoveLabel': 'Profit & Loss Value',
    'ValueMove': 'R 0.00',
    'PercentageMoveLabel': 'Profit & Loss',
    'PercentageMove': '0%',
    'PeriodMoveHeader': 'Movement on Current Holdings:'}]},
 'InvestmentTypesAndManagers': {'InvestmentTypes': [{'Key': 'Funds to Invest',
    'Value': 'R 0.00',
    'Percentage': 0.0}],
  'Managers': [{'Key': 'Self Managed', 'Value': 'R 0.00', 'Percentage': 0.0}]},
 'InvestmentSummaryItems': [{'Label': 'Total Profit / Loss on Current Holdings',
   'Value': 'R 0.

## Get Personal Transactions

In [10]:
supported_symbols: list = instance.get_personal_transactions()

print(supported_symbols)

{'TSLA': {'name': 'Tesla Inc.', 'symbol': 'TSLA', 'transactions': [{'value': 0.0688, 'date': datetime.datetime(2022, 7, 28, 0, 0)}]}, 'AAPL': {'name': 'Apple Inc.', 'symbol': 'AAPL', 'transactions': [{'value': 0.4317, 'date': datetime.datetime(2022, 7, 1, 0, 0)}]}, 'DDD': {'name': '3D Systems Corporation', 'symbol': 'DDD', 'transactions': [{'value': 8.8925, 'date': datetime.datetime(2022, 8, 5, 0, 0)}]}, 'SBSW': {'name': 'Sibanye Stillwater Ltd', 'symbol': 'SBSW', 'transactions': [{'value': 1.1265, 'date': datetime.datetime(2022, 8, 5, 0, 0)}]}}
